In [0]:
import dlt
from pyspark.sql.functions import col

@dlt.table(
    name="customer_master_xref_filtered"
)
def customer_master_xref_filtered():
    df = dlt.read("my_database.extracts.customer_master_xref")
    return df.filter(col("org") == "ERP_LN")

@dlt.table(
    name="receipt_extract"
)
def receipt_extract():
    return dlt.read("my_database.extracts.receipt_extract")

@dlt.table(
    name="int_ar_receipt_integrate"
)
def int_ar_receipt_integrate():
    cmx = dlt.read("customer_master_xref_filtered")
    re = dlt.read("receipt_extract")
    joined = cmx.join(
        re,
        (cmx.legacy_site_id == re.customer_site) &
        (cmx.legacy_location_id == re.customer_location),
        how="right"
    )
    result = joined.select(
        cmx.org, 
        cmx.customer_name, 
        cmx.fusion_location_id,
        cmx.fusion_site_id,
        re.receipt_number,
        re.receipt_amount,
        re.date   
    )
    return result